In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
# must add new line for setting the data
# x4[i,x_data[3]] = 1
training_lines = 10
vals = 4
iterations = 1200

def training_data():
    n = 0
    x = np.arange(vals+1).astype(np.int32)
    y = np.array([])
    for i in range(training_lines):
        y = np.append(y, x*n).reshape(i+1,vals+1)
        n+=1
        
    return(y)

print(training_data())

In [ ]:
data = training_data()
length = (training_lines-1)*(vals+1)*vals
def set_data():
    in_data = np.zeros([training_lines,vals,(training_lines-1)*(vals+1)])
    lab = np.zeros([training_lines,1,(training_lines-1)*(vals+1)])
    for i in range(training_lines):
        for a in range(vals):
            in_data[i][a][data[i][a]] = 1
        lab[i][0][data[i][vals]] = 1
    in_data = in_data.reshape(training_lines,1,length)
    return(in_data,lab)    

print(set_data())

In [ ]:
x = tf.placeholder(tf.float32, [None, length])
W = tf.Variable(tf.zeros([length, (training_lines-1)*(vals+1)]))
b = tf.Variable(tf.zeros([(training_lines-1)*(vals+1)]))
y = tf.matmul(x, W) + b

y_ = tf.placeholder(tf.float32, [None, (training_lines-1)*(vals+1)])

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [ ]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.4)

sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options)) 

init = tf.initialize_all_variables()
sess.run(init)
                
for _ in range(iterations):
    if _ % (iterations/training_lines) == 0:
        x_data, y_data = set_data()
        x_data = x_data[iterations/(iterations/training_lines+_)-1]
        y_data = y_data[iterations/(iterations/training_lines+_)-1]

    sess.run(train_step, feed_dict={x: x_data, y_: y_data})
        
    if _ % 100 == 0:
        print(_)
        print(sess.run(cross_entropy, feed_dict={x: x_data, y_: y_data}))

In [ ]:
test_line = np.array([])
for i in range(vals):
    val = int(input(("value",i,'?'))
    test_line = np.append(test_line, val)
def test_model():
    input_array = np.array([])
    for a in range(vals):
        test_array = np.zeros([1,(training_lines-1)*(vals+1)])
        test_array[0][test_line[a]] = 1
        input_array = np.append(input_array, test_array).reshape([1,(a+1)*(training_lines-1)*(vals+1)])
    return(input_array)        
test_model()

In [ ]:
probs = sess.run(y, feed_dict={x:test_model()})
print(probs)
print(np.argmax(probs))
answer = (test_line[1]-test_line[0])+test_line[vals-1]
print "answer =", answer
print 'probability of', answer, '=', probs[0][answer]